#### Names of people in the group

Please write the names of the people in your group in the next cell.

Joachim Maksim, joachmak

Jørgen Nordli Katralen, jorgennk

In [0]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-fb09ed1e-5b42-4f34-a4e5-9304e1deb16c/bin/python -m pip install --upgrade pip' command.


In [0]:
# Loading modules that we need
import unittest
from pyspark.sql.dataframe import DataFrame
from typing import Any

In [0]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)

users_df = load_df("/user/hive/warehouse/users")
comments_df = load_df("/user/hive/warehouse/comments")
posts_df = load_df("/user/hive/warehouse/posts")

#### Subtask 1: implenenting two helper functions
Impelment these two functions:
1. 'run_query' that gets a Spark SQL query and run it on df which is a Spark DataFrame; it returns the content of the first column of the first row of the DataFrame that is the output of the query;
2. 'run_query2' that is similar to 'run_query' but instead of one DataFrame gets two; it returns the content of the first column of the first row of the DataFrame that is the output of the query.

Note that the result of a Spark SQL query is itself a Spark DataFrame.

In [0]:
def run_query(query: "a SQL query string", df: "the DataFrame that the query will be executed on") -> Any:
    df.createOrReplaceTempView("df")
    return spark.sql(query).collect()[0][0]

def run_query2(query: "a SQL query string", df1: "DataFrame A", df2: "DataFrame B") -> Any:
    df1.createOrReplaceTempView("df1")
    df2.createOrReplaceTempView("df2")
    return spark.sql(query).collect()[0][0]

In [0]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

#### Subtask 2: writing a few queries
Write the following queries in SQL to be executed by Spark in the next cell.

1. 'q1': find the 'Id' of the most recently created post ('df' is 'posts_df') 
2. 'q2': find the number users
3. 'q3': find the 'Id' of the user who posted most number of answers
4. 'q4': find the number of questions
5. 'q5': find the display name of the user who posted most number of comments

Note that 'q1' is already available below as an example. Moreover, remmebr that Spark supports ANSI SQL 2003 so your queries have to comply with that standard.

In [0]:
q1 = "SELECT * FROM df ORDER BY CreationDate DESC limit 1"

## YOUR IMPLEMENTATION ##
q2 = "SELECT COUNT(*) FROM df"

## YOUR IMPLEMENTATION ##
q3 = "SELECT OwnerUserId, COUNT(*) AS antall FROM df GROUP BY OwnerUserId ORDER BY antall DESC"

## YOUR IMPLEMENTATION ##
q4 = "SELECT COUNT(*) FROM df WHERE PostTypeId=1"

## YOUR IMPLEMENTATION ##
q5 = "SELECT DisplayName, COUNT(*) AS antall FROM df2 JOIN df1 ON df2.UserId = df1.Id GROUP BY DisplayName ORDER BY antall DESC"

#### Subtask 3: validating the implementations by running the tests

Run the cell below and make sure that all the tests run successfully.

In [0]:
%%unittest_main
class TestTask2(unittest.TestCase):
    def test_q1(self):
        # find the id of the most recent post
        r = run_query(q1, posts_df)
        self.assertEqual(r, 95045)

    def test_q2(self):
        # find the number of the users
        r = run_query(q2, users_df)
        self.assertEqual(r, 91616)

    def test_q3(self):
        # find the user id of the user who posted most number of answers
        r = run_query(q3, posts_df)
        self.assertEqual(r, 64377)

    def test_q4(self):
        # find the number of questions
        r = run_query(q4, posts_df)
        self.assertEqual(r, 28950)

    def test_q5(self):
        # find the display name of the user who posted most number of comments
        r = run_query2(q5, users_df, comments_df)
        self.assertEqual(r, "Neil Slater")

Success

.....
----------------------------------------------------------------------
Ran 5 tests in 6.694s

OK
Out[62]: <unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please answer the following questions. Write your answer in one to two short paragraphs. Don't copy-paste; instead, write your own understanding.

1. What is the difference between 'DataFrame', 'Dataset', and 'Resilient Distributed Datasets (RDD)'? 
2. When do you suggest using RDDs instead of using DataFrames?
3. What is the main benefit of using DataSets instead of DataFrames?

Write your answers in the next cell.

1. The principal data structure of the Spark system is an acronym called RDD, which stands for Resilient Distributed Dataset. This data structure relies on transformations and actions at a low level, as previously explained in Task1. Additionally, Spark's DataFrame is a distributed data structure that is built on top of RDDs. DataFrames utilize a columnar schema, resembling a table in a relational database with named columns. However, it's essential to note that DataFrames cannot capture unstructured data, such as data streams. On the other hand, Datasets build upon DataFrames and provide an object-oriented interface that can be used for both structured and unstructured data. Both DataFrames and Datasets can automatically detect a schema, unlike RDDs, where the schema must be explicitly defined.


2. Compared to DataFrames, RDDs offer greater flexibility and can serve as a general-purpose data structure in Spark. Thus, if your data does not conform to the tabular structure of DataFrames, RDDs could be a more suitable option.


3. One of the main benefits DataSets can provide type-safety. This can help catch errors earlier rather than have them suddenly appear when you run the program. It has about the same perfomance as well.